This notebook is a continuation of TPS-Aug-Random Forest(https://www.kaggle.com/sabyasachi96/tps-aug-random-forest). 
* Recap: Well in the last notebook, the results were not that promising and also the runtime of Random Forest was too long. 
    * Error = 6.1751
    * Run time = 170 min
* My thoughts were, since the dataset is too huge a PCA can help improve the Random Forest predictions, but things didn't turn out the way i wanted to. 


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import lightgbm as lgb

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

# I'll remove the ID column of both train and test, cause they are of no use here
train.drop(columns = 'id', inplace = True)
test.drop(columns = 'id', inplace =True)

# Check
print('train columns', train.columns)
print('test columns', test.columns)

# Inference From the last notebook

Well the features were too less correlated, both positively and negatively with each other and the target.
* conducted Random forest with error 6.17 but long runtime

# Approach PCA

With the help of PCA i'll reduce the dimensions and then check their correlations before modelling and predicting

In [ ]:
# Splitting
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( train.iloc[: ,:-1], train.iloc[:,-1], 
                                                    test_size=0.2, random_state=2)
print("X-train shape", x_train.shape, "\nY-test shape" ,y_test.shape)

# Standard scaling
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_train = pd.DataFrame(ss.fit_transform(x_train))
x_test = pd.DataFrame(ss.fit_transform(x_test))

In [ ]:
# PCA
from sklearn.decomposition import PCA
pca = PCA()
pca_train = pca.fit_transform(x_train)

Now according to literature, i'll get maximum variance explained by first few Principle components. Thus haven't put any cap on the number of PC's. I'll know how many PC's needed by plotting cumulative variance 

In [ ]:
explained_variance = pca.explained_variance_ratio_

In [ ]:
plt.plot(explained_variance)
plt.xlabel('Number of components')
plt.ylabel('Explained variance')
plt.title("Scree Plot");

In [ ]:
plt.plot(np.cumsum(explained_variance))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.title("Counting PC's");

### Inference
Well the above images are disheartening, i was expecting plots that could help me getting the exact number of components. But the data lacked collinearity to a great extent and thus PCA will not be that helpful i am assuming. 
* Still I performed the Random forest on colab with the PCA data, I am not  running the code here, since it will a long time. I'll add the error value and the run time. 

In [ ]:
"""
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Initantiate model with 500 decision trees
rf = RandomForestRegressor(n_estimators=500, max_depth = 20, 
                           min_samples_split = 50, min_samples_leaf = 10,
                           verbose = 1)

# Train the model on training data
rf.fit(pca_train, y_train)
# Use the forest's predict method on the test data
y_pred = rf.predict(pca_test)
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Base line Mean Absolute Error', round(np.mean(errors), 4), 'degrees.')

"""

Error ~ 6.245;  Runtime - 122 min

# Model With Lgbm

In [ ]:
# Splitting
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( train.iloc[: ,:-1], train.iloc[:,-1], 
                                                    test_size=0.2, random_state=2)
print("X-train shape", x_train.shape, "\nY-test shape" ,y_test.shape)

P.S. I have changed the test size from 0.4 to 0.2, just to make the prediction better

In [ ]:
model = lgb.LGBMRegressor(num_iteration = 800,  learning_rate=0.03, n_estimators=150, 
                           max_depth = 5, random_state = 10 )
model.fit(x_train, y_train)

The above parameters were not randomly picked, rather its an outcome of multiple trial and errors. Since it's my first time with LGBM, next time onwards i'll be using a automated way like a loop or something else. Please suggest if you have anything. 

In [ ]:
# Use the forest's predict method on the test data
y_pred = model.predict(x_test)
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Base line Mean Absolute Error', round(np.mean(errors), 4), 'degrees.')
# Print score top check overfiting
print("train Score -", model.score(x_train,y_train))
print("test Score -", model.score(x_test,y_test))

Well the error value reduced from 6.17 to 6.16, not too great, but something atleast. 
* P.s. Do suggest other methods for a beginner to try out in such datasets or cases.

In [ ]:
submission['loss'] = model.predict(test)
submission.to_csv('submission.csv', index = False)